In [166]:
import pandas as pd
import numpy as np
import nltk
from pymorphy3 import MorphAnalyzer
from nltk.corpus import stopwords
import re

In [167]:
df_analyze = pd.read_csv('data/payments_main.tsv',index_col=False,sep='\t', names = ['id','date','payment','text'])

In [168]:
df_analyze.head()

,id,date,payment,text
0,1,07.11.2024,40500.00,За тур.поездку по договору №001 от 27.01.2023г
1,2,07.11.2024,"32600,00",За оказание услуг по договору №53Б-02746 от 23...
2,3,07.11.2024,4710-00,Оплата штрафа
3,4,07.11.2024,30900-00,Лечение по договору №Д-00359/24 от 08.03.2025
4,5,07.11.2024,13200.00,Оплата основного долга за период с 16.12.2024г...


In [169]:
df_analyze = df_analyze.drop(columns = ['id','date'])

In [170]:
df_analyze.head()

,payment,text
0,40500.00,За тур.поездку по договору №001 от 27.01.2023г
1,"32600,00",За оказание услуг по договору №53Б-02746 от 23...
2,4710-00,Оплата штрафа
3,30900-00,Лечение по договору №Д-00359/24 от 08.03.2025
4,13200.00,Оплата основного долга за период с 16.12.2024г...


#### Step-1: Remove non-cyrillic symbols

In [171]:
def remove_non_cyrillic(text):
 cyrillic_pattern = r"[^а-яА-ЯёЁ]+"
 cleaned_text = re.sub(cyrillic_pattern, " ", text)
 return cleaned_text

In [172]:
df_analyze['text'] = df_analyze['text'].apply(remove_non_cyrillic)

In [173]:
df_analyze.head()

,payment,text
0,40500.00,За тур поездку по договору от г
1,"32600,00",За оказание услуг по договору Б от
2,4710-00,Оплата штрафа
3,30900-00,Лечение по договору Д от
4,13200.00,Оплата основного долга за период с г по марта ...


#### Step-1.5 Remove joined words

def split_joined_words(text):
    splited_text = text.split(' ')
    tokens = []
    splited = False
    for token in splited_text:
        splited = False
        for i in range(1, len(token)):
            token = token.strip()
            if not token[i-1].isupper() and token[i].isupper():
                token1,token2 = token[0:i], token[i:]
                splited = True
                print(token1,token2)
                tokens.append(f' {token1} ')
                tokens.append(f' {token2} ')
        if splited is False:
            tokens.append(f' {token} ')
    return ''.join(tokens)

In [174]:
def split_joined_words(text):
    
    tokens = []
    
    def split_token(split_points, text):           
        for i in range(1,len(split_points)):
            token = text[split_points[i-1]:split_points[i]]
            tokens.append(f' {token} ')
    
    splited_text = text.split(' ')
    for token in splited_text:
        split_points = [0]
        for i in range(1, len(token)):
            token = token.strip()
            if not token[i-1].isupper() and token[i].isupper():
                split_points.append(i)
        split_points.append(len(token))
        split_token(split_points,token)
                
    return ''.join(tokens)

In [175]:
df_analyze['text'] = df_analyze['text'].apply(split_joined_words)

#### Step-2: Apply stop words and lemmatize

In [176]:
stopwords_ru = stopwords.words("russian")
morph = MorphAnalyzer()

In [177]:
def remove_stop_words(text):
    text_splited = text.split(' ')
    filtered_tokens = []
    for token in text_splited:
        if token not in stopwords_ru:
            token = token.strip()
            token = morph.normal_forms(token)[0]
            if len(token) < 2:
                continue
            filtered_tokens.append(f' {token} ')
    # if len(filtered_tokens) <= 2:
    #    return None # ( ' ' ?)
    return ''.join(filtered_tokens)

In [178]:
df_analyze['text'] = df_analyze['text'].apply(remove_stop_words)

In [179]:
df_analyze.head()

,payment,text
0,40500.00,за тур поездка договор
1,"32600,00",за оказание услуга договор
2,4710-00,оплата штраф
3,30900-00,лечение договор далее
4,13200.00,оплата основный долг период март договор...
